In [1]:
import pandas as pd

from datasets import select_all_phylas, download_all_files_ncbi
from taxonomy_processing import create_taxonomy

In [2]:
column_names = ["tax_id", "org_name", "gene_id", "current_id", "status", "symbol", "aliases", "description",
                "other_designations", "map_location", "chromosome", "genomic_nucleotide_accession.version",
                "start_position_on_the_genomic_accession", "end_position_on_the_genomic_accession", "orientation",
                "exon_count", "to_delete_1", "to_delete_2"]

df = pd.read_csv("../all_nxf1_2.txt", sep="\t", skiprows=1, names=column_names, index_col=0)
df.drop(["to_delete_1", "to_delete_2"], axis=1, inplace=True)
df.head()

,org_name,gene_id,current_id,status,symbol,aliases,description,other_designations,map_location,chromosome,genomic_nucleotide_accession.version,start_position_on_the_genomic_accession,end_position_on_the_genomic_accession,orientation,exon_count
tax_id,,,,,,,,,,,,,,,
9606,Homo sapiens,10482,0,live,NXF1,"MEX67, TAP",nuclear RNA export factor 1,nuclear RNA export factor 1|mRNA export factor...,11q12.3,11,NC_000011.10,62792130.0,62805440.0,minus,22.0
10090,Mus musculus,53319,0,live,Nxf1,"Mex67, Mvb1, Tap",nuclear RNA export factor 1,nuclear RNA export factor 1|Mex 67 homolog|mRN...,19 5.5 cM,19,NC_000085.7,8734467.0,8748274.0,plus,20.0
10116,Rattus norvegicus,59087,0,live,Nxf1,"Mex67h, Tap",nuclear RNA export factor 1,nuclear RNA export factor 1|mRNA export factor...,1q43,1,NC_086019.1,215084563.0,215097756.0,plus,21.0
8407,Rana temporaria,120917577,0,live,NXF1,NaN,nuclear RNA export factor 1,nuclear RNA export factor 1,NaN,11,NC_053499.1,120752259.0,120809664.0,plus,22.0
8364,Xenopus tropicalis,734058,0,live,nxf1,"mex67, tap",nuclear RNA export factor 1,nuclear RNA export factor 1,NaN,4,NC_030680.2,36493131.0,36532401.0,minus,23.0


In [3]:
# long-time func !!!
phyla_taxids, taxids_error = select_all_phylas(df)

ERROR taxid: 9606, Homo sapiens


In [4]:
with open("../taxonomy/all_phylas_taxonomy.tsv", "w") as outfile:
    for index, taxonomy in phyla_taxids.items():
        outfile.write(f"{index}\t{taxonomy}\n")

In [5]:
create_taxonomy("../taxonomy/all_phylas_taxonomy.tsv")

---

In [89]:
df_taxonomy = pd.read_csv("../taxonomy/all_phylas_taxonomy.tsv", sep="\t", names=["taxid", "taxonomy"], index_col=0, dtype={"taxid": int, "taxonomy": str})

In [91]:
df_amoebozoa = df_taxonomy[df_taxonomy.taxonomy.str.contains("Amoebozoa")]
df_amoebozoa

,taxonomy
taxid,
1257118,cellular organisms; Eukaryota; Amoebozoa; Disc...
261658,cellular organisms; Eukaryota; Amoebozoa; Evos...
5786,cellular organisms; Eukaryota; Amoebozoa; Evos...


In [94]:
amoebozoa_taxids = {
    "Amoebozoa": df_amoebozoa.index.tolist(),
}

In [95]:
amoebozoa_taxids

{'Amoebozoa': [1257118, 261658, 5786]}

In [96]:
download_all_files_ncbi(df, amoebozoa_taxids, phylas=list(amoebozoa_taxids.keys()))

Gene, mRNA, protein for Amoebozoa/acanthamoeba_castellanii_str._neff_0 downloaded successfully
Gene, mRNA, protein for Amoebozoa/cavenderia_fasciculata_1 downloaded successfully
Gene, mRNA, protein for Amoebozoa/dictyostelium_purpureum_2 downloaded successfully
Gene, mRNA, protein for Amoebozoa/dictyostelium_purpureum_3 downloaded successfully
Gene GenBank for Amoebozoa/acanthamoeba_castellanii_str._neff_0 downloaded successfully
Gene GenBank for Amoebozoa/cavenderia_fasciculata_1 downloaded successfully
Gene GenBank for Amoebozoa/dictyostelium_purpureum_2 downloaded successfully
Gene GenBank for Amoebozoa/dictyostelium_purpureum_3 downloaded successfully
Exons for Amoebozoa/acanthamoeba_castellanii_str._neff_0 created successfully
Exons for Amoebozoa/cavenderia_fasciculata_1 created successfully
Exons for Amoebozoa/dictyostelium_purpureum_2 created successfully
Exons for Amoebozoa/dictyostelium_purpureum_3 created successfully



In [97]:
import os

dir_name = "Amoebozoa"
ascomycota_dict = {}

for org_name in os.listdir(f"../Datasets/{dir_name}"):
    seq = ""
    with open(f"../Datasets/{dir_name}/{org_name}/ncbi_dataset/data/gene.fna", "r") as infile:
        for line in infile:
            if line.startswith(">"):
                continue
            seq += line.strip()
    ascomycota_dict[org_name] = len(seq)

In [98]:
ascomycota_dict

{'dictyostelium_purpureum_3': 1014,
 'dictyostelium_purpureum_2': 1012,
 'acanthamoeba_castellanii_str._neff_0': 2870,
 'cavenderia_fasciculata_1': 3079}

In [101]:
org_names = list(ascomycota_dict.keys())

In [102]:
org_names

['dictyostelium_purpureum_3',
 'dictyostelium_purpureum_2',
 'acanthamoeba_castellanii_str._neff_0',
 'cavenderia_fasciculata_1']

In [103]:
dir_name = "Amoebozoa"
ascomycota_dict = {}
exons_count = 0

for org_name in os.listdir(f"../Datasets/{dir_name}"):
    seq = ""
    with open(f"../Datasets/{dir_name}/{org_name}/ncbi_dataset/data/gene.fna", "r") as infile:
        for line in infile:
            if line.startswith(">"):
                exons_count += 1
    ascomycota_dict[org_name] = exons_count

In [104]:
ascomycota_dict

{'dictyostelium_purpureum_3': 1,
 'dictyostelium_purpureum_2': 2,
 'acanthamoeba_castellanii_str._neff_0': 3,
 'cavenderia_fasciculata_1': 4}

In [ ]:
org_names

In [105]:
from Scripts.data_processing import dict_align_create
from Scripts.fasta_processing import dict_align_to_fasta

dict_align = dict_align_create("Amoebozoa", org_names, "gene")

dict_align_to_fasta(dict_align, f"../Alignment/Amoebozoa_gene.fa")
dict_align_to_fasta(dict_align, f"../Alignment/Amoebozoa_gene.aln")

In [4]:
df_fungi = df_taxonomy[df_taxonomy.taxonomy.str.contains("Fungi")]

In [6]:
df_fungi

,taxonomy
taxid,
644352,cellular organisms; Eukaryota; Opisthokonta; F...
242507,cellular organisms; Eukaryota; Opisthokonta; F...
332648,cellular organisms; Eukaryota; Opisthokonta; F...
1397361,cellular organisms; Eukaryota; Opisthokonta; F...
879819,cellular organisms; Eukaryota; Opisthokonta; F...
...,...
578456,cellular organisms; Eukaryota; Opisthokonta; F...
321614,cellular organisms; Eukaryota; Opisthokonta; F...
665079,cellular organisms; Eukaryota; Opisthokonta; F...


In [8]:
df_fungi.to_csv("../fungi_taxonomy.tsv", sep="\t", header=False)

In [9]:
create_taxonomy("../taxonomy/fungi_taxonomy.tsv")

---

In [5]:
df_fungi

,taxonomy
taxid,
644352,cellular organisms; Eukaryota; Opisthokonta; F...
242507,cellular organisms; Eukaryota; Opisthokonta; F...
332648,cellular organisms; Eukaryota; Opisthokonta; F...
1397361,cellular organisms; Eukaryota; Opisthokonta; F...
879819,cellular organisms; Eukaryota; Opisthokonta; F...
...,...
578456,cellular organisms; Eukaryota; Opisthokonta; F...
321614,cellular organisms; Eukaryota; Opisthokonta; F...
665079,cellular organisms; Eukaryota; Opisthokonta; F...


In [25]:
fungi_taxids = {
    "Ascomycota": list(set(df_fungi[df_fungi.taxonomy.str.contains("Ascomycota")].index)),
    "Basidiomycota": list(set(df_fungi[df_fungi.taxonomy.str.contains("Basidiomycota")].index))
}

In [ ]:
# Ascomycota: Pezizomycotina, Saccharomycotina, Taphrinomycotina
# Basidiomycota: Agaricomycotina, Pucciniomycotina, Ustilaginomycotina, Wallemiomycotina

In [7]:
fungi_taxids_ascomycota = {
    "Pezizomycotina": list(set(df_fungi[df_fungi.taxonomy.str.contains("Pezizomycotina")].index)),
    "Saccharomycotina": list(set(df_fungi[df_fungi.taxonomy.str.contains("Saccharomycotina")].index)),
    "Taphrinomycotina": list(set(df_fungi[df_fungi.taxonomy.str.contains("Taphrinomycotina")].index)),
}

In [37]:
download_all_files_ncbi(df, fungi_taxids_ascomycota, phylas=list(fungi_taxids_ascomycota.keys()))

Gene, mRNA, protein for Pezizomycotina/gaeumannomyces_tritici_r3-111a-1_0 downloaded successfully
Gene, mRNA, protein for Pezizomycotina/cercospora_beticola_1 downloaded successfully
Gene, mRNA, protein for Pezizomycotina/cercospora_beticola_2 downloaded successfully
Gene, mRNA, protein for Pezizomycotina/sclerotinia_sclerotiorum_1980_uf-70_3 downloaded successfully
Gene, mRNA, protein for Pezizomycotina/ustilaginoidea_virens_4 downloaded successfully
Gene, mRNA, protein for Pezizomycotina/botrytis_fragariae_5 downloaded successfully
Gene, mRNA, protein for Pezizomycotina/scedosporium_apiospermum_6 downloaded successfully
Gene, mRNA, protein for Pezizomycotina/sporothrix_brasiliensis_5110_7 downloaded successfully
Gene, mRNA, protein for Pezizomycotina/penicillium_solitum_8 downloaded successfully
Gene, mRNA, protein for Pezizomycotina/thyridium_curvatum_9 downloaded successfully
Gene, mRNA, protein for Pezizomycotina/venustampulla_echinocandica_10 downloaded successfully
Gene, mRNA, p

In [5]:
fungi_taxids_basidiomycota = {
    "Agaricomycotina": list(set(df_fungi[df_fungi.taxonomy.str.contains("Agaricomycotina")].index)),
    "Pucciniomycotina": list(set(df_fungi[df_fungi.taxonomy.str.contains("Pucciniomycotina")].index)),
    "Ustilaginomycotina": list(set(df_fungi[df_fungi.taxonomy.str.contains("Ustilaginomycotina")].index)),
    "Wallemiomycotina": list(set(df_fungi[df_fungi.taxonomy.str.contains("Wallemiomycotina")].index)),
}

In [6]:
download_all_files_ncbi(df, fungi_taxids_basidiomycota, phylas=list(fungi_taxids_basidiomycota.keys()))

Gene, mRNA, protein for Agaricomycotina/marasmius_oreades_0 downloaded successfully
Gene, mRNA, protein for Agaricomycotina/heterobasidion_irregulare_tc_32_1_1 downloaded successfully
Gene, mRNA, protein for Agaricomycotina/dichomitus_squalens_lyad_421_ss1_2 downloaded successfully
Gene, mRNA, protein for Agaricomycotina/kockovaella_imperatae_3 downloaded successfully
Gene, mRNA, protein for Agaricomycotina/trichosporon_asahii_var._asahii_cbs_2479_4 downloaded successfully
Gene, mRNA, protein for Agaricomycotina/gloeophyllum_trabeum_atcc_11539_5 downloaded successfully
Gene, mRNA, protein for Agaricomycotina/tremella_mesenterica_dsm_1558_6 downloaded successfully
Gene, mRNA, protein for Agaricomycotina/laccaria_bicolor_s238n_h82_7 downloaded successfully
Gene, mRNA, protein for Agaricomycotina/serpula_lacrymans_var._lacrymans_s7.9_8 downloaded successfully
Gene, mRNA, protein for Agaricomycotina/punctularia_strigosozonata_hhb_11173_ss5_9 downloaded successfully
Gene, mRNA, protein for 

In [7]:
import os

In [62]:
dir_name = "Basidiomycota"
big_list = []

for folder_name in os.listdir(f"../Datasets/{dir_name}/"):
    print(folder_name)
    ascomycota_dict = {folder_name: []}
    for org_name in os.listdir(f"../Datasets/{dir_name}/{folder_name}"):
        seq = ""
        with open(f"../Datasets/{dir_name}/{folder_name}/{org_name}/ncbi_dataset/data/gene.fna", "r") as infile:
            for line in infile:
                if line.startswith(">"):
                    continue
                seq += line.strip()
        ascomycota_dict[folder_name].append({org_name: len(seq)})
    big_list.append(ascomycota_dict)

Agaricomycotina
Pucciniomycotina
Wallemiomycotina
Ustilaginomycotina


In [63]:
big_list

[{'Agaricomycotina': [{'mycena_indigotica_29': 2285},
   {'fibroporia_radiculosa_11': 2414},
   {'agaricus_bisporus_var._bisporus_h97_33': 2703},
   {'phanerochaete_carnosa_hhb_10118_sp_20': 2771},
   {'stereum_hirsutum_fp_91666_ss1_27': 3047},
   {'trametes_versicolor_fp_101664_ss1_36': 2413},
   {'tsuchiyaea_wingfieldii_cbs_7118_15': 2742},
   {'cutaneotrichosporon_oleaginosum_23': 2749},
   {'tremella_mesenterica_dsm_1558_6': 3286},
   {'heterobasidion_irregulare_tc_32_1_1': 2672},
   {'dichomitus_squalens_lyad_421_ss1_2': 2854},
   {'gloeophyllum_trabeum_atcc_11539_5': 2296},
   {'pleurotus_ostreatus_25': 2359},
   {'postia_placenta_mad_698_r_sb12_35': 2616},
   {'trichosporon_asahii_var._asahii_cbs_2479_4': 306},
   {'marasmius_oreades_0': 2331},
   {'suillus_fuscotomentosus_32': 3030},
   {'guyanagaster_necrorhizus_mca_3950_14': 2549},
   {'cantharellus_anzutake_13': 2371},
   {'punctularia_strigosozonata_hhb_11173_ss5_9': 5238},
   {'suillus_clintonianus_10': 2610},
   {'suillus

In [48]:
org_names = [
    list(org_name.keys())[0] for org_name in big_list[2]["Pezizomycotina"]
]

In [49]:
org_names

['orbilia_oligospora_atcc_24927_55',
 'fusarium_odoratissimum_nrrl_54006_21',
 "drepanopeziza_brunnea_f._sp._'multigermtubi'_36",
 'sphaerulina_musiva_so2202_22',
 'fusarium_vanettenii_77-13-4_14',
 'thermochaetoides_thermophila_dsm_1495_50',
 'neohortaea_acidophila_37',
 'gaeumannomyces_tritici_r3-111a-1_0',
 'coniosporium_apollinis_cbs_100218_47',
 'cercospora_beticola_2',
 'verruconis_gallopava_29',
 'aureobasidium_pullulans_exf-150_27',
 'pyricularia_pennisetigena_20',
 'zymoseptoria_tritici_ipo323_45',
 'pseudomassariella_vexata_51',
 'lindgomyces_ingoldianus_11',
 'penicillium_solitum_8',
 'botrytis_byssoidea_58',
 'botrytis_fragariae_5',
 'glarea_lozoyensis_atcc_20868_33',
 'penicillium_digitatum_18',
 'cladophialophora_immunda_13',
 'microsporum_canis_cbs_113480_19',
 'pyricularia_oryzae_70-15_38',
 'amorphotheca_resinae_atcc_22711_60',
 'aureobasidium_melanogenum_cbs_110374_28',
 'sclerotinia_sclerotiorum_1980_uf-70_3',
 'sporothrix_schenckii_1099-18_54',
 'venustampulla_echin

In [50]:
from Scripts.data_processing import dict_align_create, analyze_exons

dict_align = dict_align_create("Ascomycota/Pezizomycotina", org_names, "gene")

In [51]:
from Scripts.fasta_processing import dict_align_to_fasta

dict_align_to_fasta(dict_align, f"../Alignment/Pezizomycotina_gene.fa")
dict_align_to_fasta(dict_align, f"../Alignment/Pezizomycotina_gene.aln")

In [64]:
dir_name = "Basidiomycota"
big_list = []

for folder_name in os.listdir(f"../Datasets/{dir_name}/"):
    ascomycota_dict = {folder_name: []}
    for org_name in os.listdir(f"../Datasets/{dir_name}/{folder_name}"):
        seq = ""
        with open(f"../Datasets/{dir_name}/{folder_name}/{org_name}/ncbi_dataset/data/exons.fa", "r") as infile:
            exons_count = 0
            for line in infile:
                if line.startswith(">"):
                    exons_count += 1
        ascomycota_dict[folder_name].append({org_name: exons_count})
    big_list.append(ascomycota_dict)

In [65]:
big_list

[{'Agaricomycotina': [{'mycena_indigotica_29': 9},
   {'fibroporia_radiculosa_11': 10},
   {'agaricus_bisporus_var._bisporus_h97_33': 9},
   {'phanerochaete_carnosa_hhb_10118_sp_20': 10},
   {'stereum_hirsutum_fp_91666_ss1_27': 8},
   {'trametes_versicolor_fp_101664_ss1_36': 8},
   {'tsuchiyaea_wingfieldii_cbs_7118_15': 8},
   {'cutaneotrichosporon_oleaginosum_23': 8},
   {'tremella_mesenterica_dsm_1558_6': 8},
   {'heterobasidion_irregulare_tc_32_1_1': 10},
   {'dichomitus_squalens_lyad_421_ss1_2': 8},
   {'gloeophyllum_trabeum_atcc_11539_5': 8},
   {'pleurotus_ostreatus_25': 9},
   {'postia_placenta_mad_698_r_sb12_35': 10},
   {'trichosporon_asahii_var._asahii_cbs_2479_4': 1},
   {'marasmius_oreades_0': 9},
   {'suillus_fuscotomentosus_32': 10},
   {'guyanagaster_necrorhizus_mca_3950_14': 9},
   {'cantharellus_anzutake_13': 8},
   {'punctularia_strigosozonata_hhb_11173_ss5_9': 9},
   {'suillus_clintonianus_10': 10},
   {'suillus_discolor_30': 10},
   {'suillus_subaureus_24': 10},
   

In [75]:
analyze_exons(f"../Datasets/Basidiomycota/Agaricomycotina/mycena_indigotica_29/ncbi_dataset/data/exons.fa")

,length,coords,sequence
0,216,0-216,CCCACACACATCTACACAATGTTCTCCAGCCCAACCCCTGCGCCAG...
1,167,255-422,GTCATCGACCTCTCCTCTCTCTCTCTCGACTACTTGAAACCCACCG...
2,109,466-575,TCCAGCCTCAAGCCGTATCCTTGCTGGTAAATCAACTCGAACCCTG...
3,410,622-1032,TCCATCCAAGATGACGAAATAATTCGGAAGCATGGTCTTCAAACCC...
4,581,1086-1667,GGAGGCTCTACGTCGATTCCTCACCCTTGAGCTACTTGATCAAGAA...
5,67,1716-1783,CGCCTTCCCAGGGCATCCGCTCATTCGATAGAACCTTCATCTTGGC...
6,178,1828-2006,TGCTAGAGCCAATGGCTGGGACGTTGTCATCCTCTCCGATCAGTGG...
7,132,2058-2190,CCTGAGGCCCAGCGGCCTGCCGTCCTGGAACTCTGCGGCCGCTCCC...
8,45,2240-2285,GCGACACTACCGCCAGACGCATTCTTACGACCTATGGCATCCTAA


In [72]:
from data_processing import analyze_exons

org_names = [
    list(org_name.keys())[0] for org_name in big_list[0]["Agaricomycotina"]
]

In [73]:
org_names

['mycena_indigotica_29',
 'fibroporia_radiculosa_11',
 'agaricus_bisporus_var._bisporus_h97_33',
 'phanerochaete_carnosa_hhb_10118_sp_20',
 'stereum_hirsutum_fp_91666_ss1_27',
 'trametes_versicolor_fp_101664_ss1_36',
 'tsuchiyaea_wingfieldii_cbs_7118_15',
 'cutaneotrichosporon_oleaginosum_23',
 'tremella_mesenterica_dsm_1558_6',
 'heterobasidion_irregulare_tc_32_1_1',
 'dichomitus_squalens_lyad_421_ss1_2',
 'gloeophyllum_trabeum_atcc_11539_5',
 'pleurotus_ostreatus_25',
 'postia_placenta_mad_698_r_sb12_35',
 'trichosporon_asahii_var._asahii_cbs_2479_4',
 'marasmius_oreades_0',
 'suillus_fuscotomentosus_32',
 'guyanagaster_necrorhizus_mca_3950_14',
 'cantharellus_anzutake_13',
 'punctularia_strigosozonata_hhb_11173_ss5_9',
 'suillus_clintonianus_10',
 'suillus_discolor_30',
 'suillus_subaureus_24',
 'cryptococcus_neoformans_var._grubii_h99_17',
 'suillus_subalutaceus_22',
 'kockovaella_imperatae_3',
 'serpula_lacrymans_var._lacrymans_s7.9_8',
 'suillus_plorans_37',
 'agaricus_bisporus_v

In [87]:
exons_length_list = []

for org_name in org_names:
    df_analyze_exons = analyze_exons(f"../Datasets/Basidiomycota/Agaricomycotina/{org_name}/ncbi_dataset/data/exons.fa")
    exons_length_list.append(df_analyze_exons.length.tolist())

In [88]:
exons_length_list

[[216, 167, 109, 410, 581, 67, 178, 132, 45],
 [234, 189, 114, 242, 198, 404, 67, 181, 132, 30],
 [290, 207, 114, 239, 198, 398, 67, 193, 456],
 [362, 204, 114, 242, 195, 404, 67, 199, 132, 251],
 [277, 216, 117, 251, 605, 296, 132, 505],
 [345, 114, 242, 599, 67, 154, 132, 331],
 [213, 285, 117, 290, 213, 404, 67, 376],
 [177, 282, 117, 287, 213, 395, 67, 337],
 [426, 303, 117, 290, 213, 395, 67, 636],
 [227, 204, 117, 242, 198, 395, 67, 187, 132, 264],
 [698, 114, 242, 596, 67, 154, 132, 409],
 [237, 117, 242, 192, 395, 245, 132, 339],
 [201, 216, 353, 195, 401, 67, 190, 132, 30],
 [131, 133, 117, 242, 198, 404, 67, 181, 132, 286],
 [306],
 [195, 225, 114, 239, 195, 398, 67, 223, 147],
 [360, 210, 117, 242, 195, 395, 67, 214, 132, 311],
 [186, 291, 114, 245, 593, 67, 202, 132, 325],
 [243, 255, 368, 593, 67, 262, 136, 32],
 [666, 716, 1814, 120, 242, 201, 404, 401, 222],
 [201, 213, 117, 242, 195, 395, 67, 211, 132, 30],
 [356, 210, 117, 242, 195, 395, 67, 214, 132, 427],
 [392, 213,

In [144]:
df_taxonomy[df_taxonomy.taxonomy.str.contains("Viruses")]

,taxonomy
taxid,
11676,Viruses; Riboviria; Pararnavirae; Artvervirico...
2697049,Viruses; Riboviria; Orthornavirae; Pisuviricot...
10298,Viruses; Duplodnaviria; Heunggongvirae; Peplov...
333760,Viruses; Monodnaviria; Shotokuvirae; Cossaviri...
10376,Viruses; Duplodnaviria; Heunggongvirae; Peplov...
211044,Viruses; Riboviria; Orthornavirae; Negarnaviri...
10600,Viruses; Monodnaviria; Shotokuvirae; Cossaviri...
1335626,Viruses; Riboviria; Orthornavirae; Pisuviricot...
10335,Viruses; Duplodnaviria; Heunggongvirae; Peplov...
